In [50]:
from pyspark import SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics

In [16]:
sc = SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=yarn) created by __init__ at <ipython-input-2-4443dd5159b0>:1 

In [90]:
spam = sc.textFile("gs://cs660_djd378/spam.csv")
ham = sc.textFile("gs://cs660_djd378/ham.csv")

spam_words = spam.map(lambda x: x.lower().split())
ham_words = ham.map(lambda x: x.lower().split())

In [91]:
tf = HashingTF(numFeatures = 200)
spam_features = tf.transform(spam_words)
ham_features = tf.transform(ham_words)

In [92]:
spam_samples = spam_features.map(lambda features:LabeledPoint(1, features))
ham_samples = ham_features.map(lambda features:LabeledPoint(0, features))

In [93]:
samples = spam_samples.union(ham_samples)
[training_data, test_data] = samples.randomSplit([0.8, 0.2])
training_data.cache()
test_data.cache()

PythonRDD[1093] at RDD at PythonRDD.scala:52

In [94]:
%%time
algo = NaiveBayes()
model = algo.train(training_data)

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 1.36 s


In [95]:
spam_test = tf.transform("Hello For inquiries about our philanthropy donation: Please email ::cdcheinkencharity@att.net Heineken Family Charity".split())
print(model.predict(spam_test))

ham_test = tf.transform("I'm testing to see if this works".split())
print(model.predict(ham_test))

1.0
0.0


In [96]:
%%time
preds_and_label = test_data.map(lambda lp: (float(model.predict(lp.features)), lp.label))

# Instantiate metrics object
metrics = MulticlassMetrics(preds_and_label)

# Overall statistics
print("Recall = %s" % metrics.recall())
print("Precision = %s" %  metrics.precision())
print("Specificity  = %s" % (1 - metrics.falsePositiveRate(0.0)))

Recall = 0.928255093003
Precision = 0.928255093003
Specificity  = 0.680981595092
CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 789 ms


In [ ]:
sc.stop()